In [2]:
import pandas as pd
from pathlib import Path  
from itertools import product
import numpy as np
import re


# TODO
# do a matching arrangement with every tail and every head with this new method. 
# add these to the list of movies


In [31]:
def clean_title(title):
    return re.sub(r'[^\w]', '', title).lower()

filePath = Path('data/popularmovies.csv')  
df = pd.read_csv(filePath) 
df = df.loc[:, df.columns.intersection(['title','overview', 'popularity'])]
df.sort_values('popularity')
df['clean_title'] = df.title.apply(clean_title)


def find_heads(title, titles_list):
    matches = []
    for t in titles_list:
        if t != title: 
            for i in range(4,9):
                if t.endswith(title[:i]):
                    if not t.endswith(title):
                        matches.append(t)
                    break
    return matches

def find_tails(title, titles_list):
    matches = []
    for t in titles_list:
        if t != title: 
            for i in range(4,9):
                if title.endswith(t[:i]):
                    if not title.endswith(t):
                        matches.append(t)
                    break
    return matches
    
def generate_centipedes(row):
    return list(product(row['heads'], [row['clean_title']], row['tails']))

df['tails'] = df['clean_title'].apply(lambda x: find_tails(x, df['clean_title'].tolist()))
df['tail_len'] = df['tails'].apply(len)
df['heads'] = df['clean_title'].apply(lambda x: find_heads(x, df['clean_title'].tolist()))
df['head_len'] = df['heads'].apply(len)
df['naiive_score'] = df['tail_len'] * df['head_len']
df['full_centipede'] = df.apply(generate_centipedes, axis=1)
df

,title,overview,popularity,clean_title,tails,tail_len,heads,head_len,naiive_score,full_centipede
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,186.429,theshawshankredemption,[],0,[],0,0,[]
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",249.306,thegodfather,"[theresstilltomorrow, therewillbeblood, thered...",20,[],0,0,[]
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,89.247,thegodfatherpartii,[],0,[],0,0,[]
3,Schindler's List,The true story of how businessman Oskar Schind...,126.803,schindlerslist,[],0,[],0,0,[]
4,12 Angry Men,The defense and the prosecution have rested an...,76.259,12angrymen,[],0,[],0,0,[]
...,...,...,...,...,...,...,...,...,...,...
1975,Batman Beyond: Return of the Joker,"The Joker is back with a vengeance, and Neo-Go...",20.815,batmanbeyondreturnofthejoker,[],0,[thebatman],1,0,[]
1976,Crossroads,A wanna-be blues guitar virtuoso seeks a long-...,19.169,crossroads,[],0,[],0,0,[]
1977,Carrie,Withdrawn and sensitive teen Carrie White face...,29.995,carrie,[],0,[],0,0,[]
1978,Phantom Thread,"In 1950s London, a renowned dressmaker's metic...",24.640,phantomthread,"[threads, readyplayerone]",2,[],0,0,[]


In [34]:


df_centipede = df.explode('full_centipede', ignore_index=True)
df_centipede = df_centipede.loc[df_centipede['naiive_score'] > 0]
df_centipede[['head', 'body', 'tail']] = pd.DataFrame(df_centipede['full_centipede'].tolist(), index=df_centipede.index)
df_centipede = df_centipede.drop(columns =['tail_len',	'head_len','naiive_score', 'heads', 'tails',  'clean_title',])
df_centipede = df_centipede.rename(columns={"popularity": "body_pop", "overview": "body_view",  'title': 'body_title'})

df_popularity = df[['title','overview','clean_title', 'popularity']]

df_centipede = df_centipede.merge(df_popularity,
                                  left_on='tail',
                                  right_on='clean_title',
                                  how='left'
                                 ).rename(columns={'popularity': 'tail_pop', 'overview': "tail_overview", }
                                         ).drop(columns = ["clean_title"])
df_centipede = df_centipede.merge(df_popularity,
                                  left_on='head',
                                  right_on='clean_title',
                                  how='left'
                                 ).rename(columns={'popularity': 'head_pop', 'overview': "head_overview", }
                                         ).drop(columns = ["clean_title"])
df_centipede
df_centipede['least_pop'] = df_centipede[['head_pop', 'body_pop', 'tail_pop']].min(axis=1)
df_centipede.sort_values('least_pop', ascending = False)



,body_title,body_view,body_pop,full_centipede,head,body,tail,title_x,tail_overview,tail_pop,title_y,head_overview,head_pop,least_pop
258,Everything Everywhere All at Once,An aging Chinese immigrant is swept up in an i...,74.719,"(toalltheboysalwaysandforever, everythingevery...",toalltheboysalwaysandforever,everythingeverywhereallatonce,onceuponatimeinhollywood,Once Upon a Time... in Hollywood,"Los Angeles, 1969. TV star Rick Dalton, a stru...",72.894,To All the Boys: Always and Forever,Senior year of high school takes center stage ...,88.107,72.894
833,Black Panther,"King T'Challa returns home to the reclusive, t...",86.777,"(meetjoeblack, blackpanther, therevenant)",meetjoeblack,blackpanther,therevenant,The Revenant,"In the 1820s, a frontiersman, Hugh Glass, sets...",77.835,Meet Joe Black,"William Parrish, media tycoon and loving fathe...",69.764,69.764
256,Everything Everywhere All at Once,An aging Chinese immigrant is swept up in an i...,74.719,"(toalltheboysalwaysandforever, everythingevery...",toalltheboysalwaysandforever,everythingeverywhereallatonce,onceuponatimeinthewest,Once Upon a Time in the West,As the railroad builders advance unstoppably t...,57.893,To All the Boys: Always and Forever,Senior year of high school takes center stage ...,88.107,57.893
402,Deadpool,The origin story of former Special Forces oper...,101.872,"(shaunofthedead, deadpool, deadpoolwolverine)",shaunofthedead,deadpool,deadpoolwolverine,Deadpool & Wolverine,A listless Wade Wilson toils away in civilian ...,587.267,Shaun of the Dead,"Shaun lives a supremely uneventful life, which...",51.735,51.735
30,Dead Poets Society,"At an elite, old-fashioned boarding school in ...",68.678,"(shaunofthedead, deadpoetssociety, societyofth...",shaunofthedead,deadpoetssociety,societyofthesnow,Society of the Snow,"On October 13, 1972, Uruguayan Air Force Fligh...",58.762,Shaun of the Dead,"Shaun lives a supremely uneventful life, which...",51.735,51.735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,"Love, Simon","Everyone deserves a great love story, but for ...",23.190,"(hairlove, lovesimon, simonewomanofthecentury)",hairlove,lovesimon,simonewomanofthecentury,Simone: Woman of the Century,Simone Veil's life story through the pivotal e...,13.424,Hair Love,When dad has to unexpectedly step in for mom t...,4.939,4.939
551,Love and Death,"In czarist Russia, a neurotic soldier and his ...",11.119,"(hairlove, loveanddeath, deathnoterelight1visi...",hairlove,loveanddeath,deathnoterelight1visionsofagod,Death Note Relight 1: Visions of a God,"A recap of Death Note episodes 1–26, with new ...",206.595,Hair Love,When dad has to unexpectedly step in for mom t...,4.939,4.939
670,Home Alone,Eight-year-old Kevin McCallister makes the mos...,3.830,"(adogswayhome, homealone, lonesurvivor)",adogswayhome,homealone,lonesurvivor,Lone Survivor,Four Navy SEALs on a covert mission to neutral...,48.492,A Dog's Way Home,"The adventure of Bella, a dog who embarks on a...",25.240,3.830
669,Home Alone,Eight-year-old Kevin McCallister makes the mos...,3.830,"(spidermanfarfromhome, homealone, lonesurvivor)",spidermanfarfromhome,homealone,lonesurvivor,Lone Survivor,Four Navy SEALs on a covert mission to neutral...,48.492,Spider-Man: Far From Home,Peter Parker and his friends go on a summer tr...,85.514,3.830


,title,overview,popularity,clean_title,tails,tail_len,heads,head_len,naiive_score
731,Mother,"A mother lives quietly with her son. One day, ...",20.589,mother,"[theresstilltomorrow, therewillbeblood, thered...",18,"[thesecondmother, allaboutmymother, ikilledmym...",3,54
219,There Will Be Blood,"Ruthless silver miner, turned oil prospector, ...",37.479,therewillbeblood,"[blooddiamond, bloodandbone, bloodandblacklace]",3,"[thegodfather, thefather, thesecondmother, whe...",15,45
1111,Happy Together,A gay couple from Hong Kong takes a trip to Ar...,18.345,happytogether,"[theresstilltomorrow, therewillbeblood, thered...",18,"[boburnhammakehappy, askmeifimhappy]",2,36
648,Everything Everywhere All at Once,An aging Chinese immigrant is swept up in an i...,74.719,everythingeverywhereallatonce,"[onceuponatimeinamerica, onceuponastudio, once...",8,"[lilya4ever, thegreatestbeerrunever, toalltheb...",4,32
1763,The Remains of the Day,A rule-bound head butler's world of manners an...,20.216,theremainsoftheday,"[thedayofthejackal, thedaytheearthstoodstill]",2,"[thegodfather, thefather, thesecondmother, whe...",15,30
...,...,...,...,...,...,...,...,...,...
680,Hustle,After discovering a once-in-a-lifetime player ...,25.717,hustle,[],0,[kungfuhustle],1,0
678,Misery,"After an accident, acclaimed novelist Paul She...",30.041,misery,[],0,[],0,0
677,The Chaser,Joong-ho is a dirty detective turned pimp in f...,23.308,thechaser,[],0,[],0,0
676,Do the Right Thing,"Salvatore ""Sal"" Fragione is the Italian owner ...",15.419,dotherightthing,[],0,[],0,0


In [229]:
# clean title row
def clean_title(title):
    words = title.split()
    # if (words[0].lower() == 'the'):
    #     return " ".join(words[1:]).lower()
    return title.lower()

df['clean_title'] = df.title.apply(clean_title)
df['title_len'] = df.title.str.split().apply(len)

colNames = ['title','overview','popularity','clean_title','title_len']
df = df.loc[df.title_len > 1, colNames]

df['first_word'] = df.clean_title.str.split().str[0]
df['last_word'] = df.clean_title.str.split().str[-1]

In [230]:
# Do a bland Movie Centipede with just complete words
# add two columns. One for head and one for tail
lastWords = df.groupby('last_word')['clean_title'].apply(list).to_dict()
firstWords = df.groupby('first_word')['clean_title'].apply(list).to_dict()

df['heads'] = df['last_word'].map(lambda x: firstWords.get(x,[]))
df['tails'] = df['first_word'].map(lambda x: lastWords.get(x,[]))

df['tail_length'] = df['tails'].apply(len)
df['head_length'] = df['heads'].apply(len)
df['naiive_score'] = df.tail_length * df.head_length

df = df.sort_values("naiive_score", ascending = False)


,title,overview,popularity,clean_title,title_len,first_word,last_word,heads,tails,tail_length,head_length,naiive_score
208,Transformers One,The untold origin story of Optimus Prime and M...,321.281,transformers one,2,transformers,one,"[one flew over the cuckoo's nest, one hundred ...",[],0,7,0


In [233]:
#for every row

def generate_centipedes(row):
    return list(product(row['tails'], [row['clean_title']], row['heads']))
    
df['full_centipede'] = df.apply(generate_centipedes, axis=1)
df_centipede = df.explode('full_centipede', ignore_index=True)
df_centipede = df_centipede.loc[df_centipede.naiive_score > 0]


df_centipede[['head', 'body', 'tail']] = pd.DataFrame(df_centipede['full_centipede'].tolist(), index=df_centipede.index)
df_centipede = df_centipede.drop(columns =['tail_length',	'head_length','naiive_score', 'heads', 'tails', 'first_word', 'last_word', 'title_len', 'clean_title', 'title'])
df_centipede = df_centipede.rename(columns={"popularity": "body_pop", "overview": "body_view"})
df_centipede = df_centipede[['head', 'body', 'tail',"body_pop", "body_view"]]
df_popularity = df.drop(columns = ['title','overview','title_len','first_word','last_word','heads','tails','tail_length','head_length','naiive_score', 'full_centipede'])

df_centipede = df_centipede.merge(df_popularity, left_on='head', right_on='clean_title', how='left', suffixes=("", "")).rename(columns={'popularity': 'head_pop'}).drop(columns = ["clean_title"])
df_centipede = df_centipede.merge(df_popularity, left_on='tail', right_on='clean_title', how='left').rename(columns={'popularity': 'tail_pop'}).drop(columns = ["clean_title"])
# df_centipede = df_centipede.drop(columns = ['full_centipede'])
df_centipede['least_pop'] = df_centipede[['head_pop', 'body_pop', 'tail_pop']].min(axis=1)

# make a new df_for each instance that includes the head, body, tail, head_pop, body_pop, tail_pop

In [234]:
df_centipede.sort_values("least_pop", ascending = False)

# I want to sort by popularity of lowest

,head,body,tail,body_pop,body_view,head_pop,tail_pop,least_pop
270,shaun of the dead,dead poets society,society of the snow,68.678,"At an elite, old-fashioned boarding school in ...",51.735,58.762,51.735
321,apocalypse now,now you see me,me before you,51.322,An FBI agent and an Interpol detective track a...,57.104,103.297,51.322
218,puss in boots: the last wish,wish dragon,dragon ball z: bardock - the father of goku,42.563,Determined teen Din is longing to reconnect wi...,143.582,51.475,42.563
215,puss in boots: the last wish,wish dragon,dragon ball super: super hero,42.563,Determined teen Din is longing to reconnect wi...,143.582,45.191,42.563
169,the skin i live in,in the mood for love,love don't co$t a thing,42.924,"In 1962 Hong Kong, two neighbors form a strong...",42.096,62.655,42.096
...,...,...,...,...,...,...,...,...
194,"batman: the long halloween, part one",one piece film: strong world,world of tomorrow,20.353,"20 years after his escape from Impel Down, the...",13.690,5.043,5.043
362,the skin i live in,in this corner of the world,world of tomorrow,11.353,"Japan, 1943, during World War II. Young Suzu l...",42.096,5.043,5.043
361,let the right one in,in this corner of the world,world of tomorrow,11.353,"Japan, 1943, during World War II. Young Suzu l...",23.471,5.043,5.043
175,hair love,love and death,death note relight 1: visions of a god,11.119,"In czarist Russia, a neurotic soldier and his ...",4.939,206.595,4.939


In [256]:
import string
translator = str.maketrans('', '', string.punctuation)

# Remove punctuation
clean_text = s.translate(translator)
print(clean_text)

AttributeError: module 'string' has no attribute 'maketrans'